In [1]:
# jupyter测试
print('hello Jupyter~~~!')

hello Jupyter~~~!


In [1]:
# 获取mnist数据集，并加载到内存中
import mxnet as mx
import gzip
import struct
import numpy as np
def read_data(label_url, image_url):
    with gzip.open(label_url) as flbl:
        struct.unpack(">II", flbl.read(8))
        label = np.frombuffer(flbl.read(), dtype=np.int8)
    with gzip.open(image_url, 'rb') as fimg:
        _, _, rows, cols = struct.unpack(">IIII", fimg.read(16))
        image = np.frombuffer(fimg.read(), dtype=np.uint8).reshape(len(label), rows, cols)
        image = image.reshape(image.shape[0], 1, 28, 28).astype(np.float32)/255
    return (label, image)

def get_mnist_data():
    # changed to mxnet.io for more stable hosting
    # path = 'http://yann.lecun.com/exdb/mnist/'
    path = 'C:/Users/zhaoyu/Documents/'
    (train_lbl, train_img) = read_data(
        path+'train-labels-idx1-ubyte.gz', path+'train-images-idx3-ubyte.gz')
    (test_lbl, test_img) = read_data(
        path+'t10k-labels-idx1-ubyte.gz', path+'t10k-images-idx3-ubyte.gz')
    return {'train_data':train_img, 'train_label':train_lbl,
            'test_data':test_img, 'test_label':test_lbl}

mnist = get_mnist_data()


In [2]:
# 初始化MNIST数据集的数据迭代器，一个用于训练一个用于测试
batch_size = 100
train_iter = mx.io.NDArrayIter(mnist['train_data'],mnist['train_label'],batch_size,shuffle=True)
val_iter = mx.io.NDArrayIter(mnist['test_data'],mnist['test_label'],batch_size)


In [3]:
data = mx.sym.var('data')
# first conv layer
conv1 = mx.sym.Convolution(data=data, kernel=(5,5), num_filter=20)
tanh1 = mx.sym.Activation(data=conv1, act_type="tanh")
pool1 = mx.sym.Pooling(data=tanh1, pool_type="max", kernel=(2,2), stride=(2,2))
# second conv layer
conv2 = mx.sym.Convolution(data=pool1, kernel=(5,5), num_filter=50)
tanh2 = mx.sym.Activation(data=conv2, act_type="tanh")
pool2 = mx.sym.Pooling(data=tanh2, pool_type="max", kernel=(2,2), stride=(2,2))
# first fullc layer
flatten = mx.sym.flatten(data=pool2)
fc1 = mx.symbol.FullyConnected(data=flatten, num_hidden=500)
tanh3 = mx.sym.Activation(data=fc1, act_type="tanh")
# second fullc
fc2 = mx.sym.FullyConnected(data=tanh3, num_hidden=10)
# softmax loss
lenet = mx.sym.SoftmaxOutput(data=fc2, name='softmax')

In [4]:
#使用SGD梯度下降方法开始训练
import logging
logging.getLogger().setLevel(logging.DEBUG)  # logging to stdout
# create a trainable module on GPU 0
lenet_model = mx.mod.Module(symbol=lenet, context=mx.cpu())

# 创建检查点，保存模型
model_prefix = 'mx_mlp'
checkpoint = mx.callback.do_checkpoint(model_prefix)

# train with the same
lenet_model.fit(train_iter,
                eval_data=val_iter,
                optimizer='sgd',
                optimizer_params={'learning_rate':0.1},
                eval_metric='acc',
                batch_end_callback = mx.callback.Speedometer(batch_size, 100),
                num_epoch=10,
                epoch_end_callback=checkpoint)






INFO:root:Epoch[0] Batch [100]	Speed: 2091.10 samples/sec	accuracy=0.108515


INFO:root:Epoch[0] Batch [200]	Speed: 2290.46 samples/sec	accuracy=0.114900


INFO:root:Epoch[0] Batch [300]	Speed: 1940.34 samples/sec	accuracy=0.110800


INFO:root:Epoch[0] Batch [400]	Speed: 1943.89 samples/sec	accuracy=0.111800


INFO:root:Epoch[0] Batch [500]	Speed: 2234.77 samples/sec	accuracy=0.115800


...

INFO:root:Epoch[9] Batch [500]	Speed: 1906.05 samples/sec	accuracy=0.990600


INFO:root:Epoch[9] Train-accuracy=0.992424


INFO:root:Epoch[9] Time cost=29.782


INFO:root:Saved checkpoint to "mx_mlp-0010.params"


INFO:root:Epoch[9] Validation-accuracy=0.988300


In [10]:
#使用模型进行预测
test_iter = mx.io.NDArrayIter(mnist['test_data'], None, batch_size)
prob = lenet_model.predict(test_iter)
test_iter = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], batch_size)
# predict accuracy for lenet
acc = mx.metric.Accuracy()
lenet_model.score(test_iter, acc)
print("Accuracy score is %f" % (acc.get()[1]))
assert acc.get()[1] > 0.98

Accuracy score is 0.988300
EvalMetric: {'accuracy': 0.98829999999999996}


In [32]:
import  numpy as np
import pandas as pd
data = pd.DataFrame(prob.asnumpy())

writer = pd.ExcelWriter('A.xlsx')		# 写入Excel文件
data.to_excel(writer, 'page_1', float_format='%.5f')		# ‘page_1’是写入excel的sheet名
writer.save()
writer.close()

DEBUG:matplotlib:$HOME=C:\Users\zhaoyu


DEBUG:matplotlib:matplotlib data path D:\anacoda\lib\site-packages\matplotlib\mpl-data


DEBUG:matplotlib:loaded rc file D:\anacoda\lib\site-packages\matplotlib\mpl-data\matplotlibrc


DEBUG:matplotlib:matplotlib version 2.2.2


DEBUG:matplotlib:interactive is False


DEBUG:matplotlib:platform is win32


DEBUG:matplotlib:loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'nt', 'winreg', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'encodings.gbk', '_codecs_cn', '_multibytecodec', 'site', 'os', 'errno', 'stat', '_stat', 'ntpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'sysconfig', 'types', 'functools', '_functools', 'collections', 'operator', '_operator', 'keyword', 'heapq', '_heapq', 'itertools', 'reprlib', '_collections', 'weakref', 'collections.abc', 'importlib', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib.util', 'importlib.abc', 'importlib.machinery', 'contextlib', 'mpl_toolkits', 'google', 'runpy', 'pkgutil', 'ipykernel', 'ipykernel._version', 'ipykernel.connect', '__future__', 'json', 'js